# Graph Neural Networks Baseline
### without timeseries as node features


In [55]:
import numpy as np
import pickle
import pandas as pd 
import matplotlib.pyplot as plt

In [56]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

In [3]:
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool
from torch_geometric.data import Data, DataLoader

In [4]:
from torch.utils.tensorboard import SummaryWriter

In [5]:
from sklearn.model_selection import StratifiedKFold

## Load data

In [6]:
DATA_FOLDER = '../data'
PICKLE_FOLDER = '../pickles'

In [7]:
df_metadata = pd.read_csv(f'{DATA_FOLDER}/patients-cleaned.csv', index_col=0)

In [8]:
df_metadata.head(2)

,age,sex,target
0,24.750,1,0
1,27.667,1,0


### Select connectivity dataset

In [17]:
THRESHOLD = 0.1                         # 0.01, 0.05, 0.1, 0.15
N = 5                                   # 3, 5, 7, 10, 15, 20, 40
EDGE_FEATURES = 'binary'                # 'binary', 'real'
CORR_TYPE = 'pearson'                   # 'pearson', 'spearman', 'partial-pearson'
THRESHOLD_METHOD = 'abs-sample-diff'    # 'abs-sample-diff', 'abs-group-avg-diff'
THRESHOLD_TYPE = 'large'                # 'min', 'max' or for kNN 'small', 'large'
KNN = True                              # Whether all or only top N neigbors are taken

In [18]:
fc_folder = f'{PICKLE_FOLDER}/fc-{CORR_TYPE}{"-knn-" if KNN else ""}{THRESHOLD_METHOD}'

In [19]:

fc_file = f'{fc_folder}/{THRESHOLD_TYPE}-{f"knn-{N}" if KNN else f"th-{THRESHOLD}"}-{EDGE_FEATURES}.pickle'

In [20]:
with open(fc_file, 'rb') as f:
    edge_index_matrix = pickle.load(f)

In [21]:
edge_index_matrix.shape

(190, 90, 90)

In [22]:
total_samples, total_brain_regions, _ = edge_index_matrix.shape

## Split data

In [23]:
with open(f'{PICKLE_FOLDER}/test-indices.pickle', 'rb') as f:
    test_indices = pickle.load(f)

In [24]:
train_indices = list(set(range(total_samples)) - set(test_indices))

In [25]:
train_targets = df_metadata.iloc[train_indices]["target"].reset_index(drop=True)

In [26]:
print(f'Train set size: {len(train_indices)}')
print(f'Test set size: {len(test_indices)}')

Train set size: 140
Test set size: 50


## Prepare data

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### `Data` object fields

- `data.x`: Node feature matrix with shape `[num_nodes, num_node_features]`

- `data.edge_index`: Graph connectivity in COO format with shape `[2, num_edges]` and type `torch.long`

- `data.edge_attr`: Edge feature matrix with shape `[num_edges, num_edge_features]`

- `data.y`: Target to train against (may have arbitrary shape), e.g., node-level targets of shape `[num_nodes, *]` or graph-level targets of shape `[1, *]`

- `data.pos`: Node position matrix with shape `[num_nodes, num_dimensions]`

In [29]:
dataset = [Data(
    x=torch.ones([90, 1], dtype=torch.float32),  
    edge_index=torch.from_numpy(np.asarray(np.nonzero(edge_index_matrix[i]))).to(torch.int64),
    y=torch.tensor([target], dtype=torch.int64)
).to(device) for target, i in zip(train_targets, train_indices)]

In [30]:
print('Data object')
print(f'Edge index: {dataset[0].edge_index.shape}')
print(f'Node features: {dataset[0].x.shape}')
print(f'Target: {dataset[0].y.shape}')

Data object
Edge index: torch.Size([2, 90])
Node features: torch.Size([90, 1])
Target: torch.Size([1])


## Define model

In [31]:
class BaselineGCN(torch.nn.Module):
    
    def __init__(self, hidden_channels):
        super(BaselineGCN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(1, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

        self.fc1 = Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # 1. Obtain node embeddings.
        # Two as in Kipf, Welling paper (2 cons, ReLU).
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)

        # 2. Readout layer.
        x = global_add_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier.
        x = self.fc1(x)     # CELoss already incorporates `log_softmax`.
        
        return x

In [32]:
# Architecture.
BaselineGCN(hidden_channels=8)

BaselineGCN(
  (conv1): GCNConv(1, 8)
  (conv2): GCNConv(8, 8)
  (fc1): Linear(in_features=8, out_features=2, bias=True)
)

## Evaluation

In [33]:
def evaluation_metrics(predicted, labels):
    pred_positives = predicted == 1
    label_positives = labels == 1

    tp = (pred_positives & label_positives).sum().item()
    tn = (~pred_positives & ~label_positives).sum().item()
    fp = (pred_positives & ~label_positives).sum().item()
    fn = (~pred_positives & label_positives).sum().item()

    return tp, tn, fp, fn

## Train model

In [34]:
NUM_FOLDS = 7    # 140 = (120 + 20)

In [35]:
skf = StratifiedKFold(n_splits=NUM_FOLDS, random_state=42, shuffle=True)

In [45]:
# Settings.
EPOCHS = 100
LR = 0.001
MOMENTUM = 0.5
OPTIMIZER = 'sgd'
LOSS = 'ce'
BATCH_SIZE = 4

HIDDEN_CHANNELS = 64

VALIDATE_FREQ = 10

In [46]:
# We only assess the first fold during model search.
train_index, val_index = next(skf.split(np.zeros(len(train_targets)), train_targets))

In [47]:
# Prepare data.
X_train = [dataset[i] for i in train_index]
X_val = [dataset[i] for i in val_index]

trainloader = DataLoader(X_train, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(X_val, batch_size=BATCH_SIZE, shuffle=False)

In [50]:
# Experiment.
EXP_ID = 1

In [53]:
# Init TB writer.
foldername = f'id={EXP_ID:03d},fold=XXX,bs={BATCH_SIZE},e={EPOCHS},lr={LR},mom={MOMENTUM},opt={OPTIMIZER},loss={LOSS}'
writer = SummaryWriter(f"../runs/{foldername}")

# Init model.
net = BaselineGCN(hidden_channels=HIDDEN_CHANNELS).to(device)
optimizr = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM)
criterion = torch.nn.CrossEntropyLoss()

# Save architecture.
with open(f"../runs/{foldername}/architecture", 'w') as f:
    f.write(net.__str__())

In [54]:
# Train.
for epoch in range(EPOCHS):
    running_loss = 0.
    
    for data in trainloader:
        net.train()
        optimizr.zero_grad()

        outputs = net(data)
        
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizr.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(trainloader)
    writer.add_scalar('training loss', epoch_loss, epoch)

    running_loss = 0.

    # Evaluate epoch.
    tp, tn, fp, fn = 0, 0, 0, 0
    total = 0

    for data in valloader:
        net.eval()
        optimizr.zero_grad()

        outputs = net(data)
        
        loss = criterion(outputs, data.y)  # Compute the loss.
        running_loss += loss.item()

        if (epoch+1) % VALIDATE_FREQ == 0:
            predicted = outputs.argmax(dim=1)
            labels = data.y.view(-1)

            # Update.
            tp_, tn_, fp_, fn_ = evaluation_metrics(predicted, labels)
            tp += tp_; tn += tn_; fp += fp_; fn += fn_
            total += data.y.size(0)

    epoch_loss = running_loss / len(valloader)
    writer.add_scalar('validation loss', epoch_loss, epoch)
    
    if (epoch+1) % VALIDATE_FREQ == 0:
        writer.add_scalar('validation accuracy', (tp + tn) / total, epoch)
        writer.add_scalar('validation precision', tp / (tp + fp), epoch)
        writer.add_scalar('validation recall', tp / (tp + fn), epoch)

print('Finished training')

Finished training
